In [1]:
import sys
sys.dont_write_bytecode = True

# for debug print
def chkprint(*args):
    from inspect import currentframe
    names = {id(v):k for k,v in currentframe().f_back.f_locals.items()}
    print(', '.join(names.get(id(arg),'???')+' : '+str(type(arg))+' = '+repr(arg) for arg in args))

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import time

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# 時間計測 start
start = time.time()

# ワイン白データ読み込み
wine = pd.read_csv("winequality-white.csv", sep=";", encoding="utf-8")

# データをラベルとデータに分離 
y = wine["quality"]
x = wine.drop("quality", axis=1)

# y のラベルを付け直す 
newlist = []
for v in list(y):
    if v <= 4:
        newlist += [0]
    elif v <=7:
        newlist += [1]
    else:
        newlist += [2]
y = newlist

# 学習用とテスト用に分離する 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, train_size = 0.8, shuffle = True)

# グリッドサーチで利用するパラメータを指定 --- (*1)
parameters = [
    {"C": [1, 10, 100, 1000], "kernel":["linear"]},
    {"C": [1, 10, 100, 1000], "kernel":["rbf"], "gamma":[0.001, 0.0001]},
    {"C": [1, 10, 100, 1000], "kernel":["sigmoid"], "gamma": [0.001, 0.0001]}
]

# グリッドサーチを行う --- (*2)
kfold_cv = KFold(n_splits=5, shuffle=True)
clf = GridSearchCV( SVC(), parameters, cv=kfold_cv)
clf.fit(x_train, y_train)
print("最適なパラメータ = ", clf.best_estimator_)

# 最適なパラメータで評価 --- (*3)
y_pred = clf.predict(x_test)
print("評価時の正解率 = " , accuracy_score(y_test, y_pred))

# 時間計測 finish
elapsed_time = time.time() - start
print ("\n elapsed_time:{0}".format(elapsed_time) + "[sec]")